In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd /content/drive/MyDrive/새싹프로젝트_텍스트

/content/drive/MyDrive/새싹프로젝트_텍스트


In [5]:
ls

'circulus koelectra-polite-v1.ipynb'   Mecab-ko-for-Google-Colab/
 concat.csv                            model_trained.pt
 crwaling.ipynb                        modified_mecab_add_kobert_with_wandb_sweep.ipynb
 data_processed.csv                   'monologg kobert.ipynb'
 df_pre.csv                            pt_dataset/
 han_article.csv                       Untitled
 han_rename.csv                        wandb/
 huggingface.ipynb                     wandb_use_news.ipynb
 KoELECTRA.ipynb                       셀레니움을_쓰지않았을때.html
 mecab_add_kobert.ipynb                한겨레전처리.ipynb


In [6]:
!pip install transformers torch pandas tqdm



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 6.0 MB/s eta 0:00:00


# 데이터 불러오기 ( 지금은 han 데이터만 씀 -> 나중에 전체꺼 오면 바꿔서 실행.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

from transformers import ElectraForSequenceClassification, ElectraTokenizer

# 사전 훈련된 모델 로드
model = ElectraForSequenceClassification.from_pretrained("copycats/koelectra-base-v3-generalized-sentiment-analysis")

# classifier의 out_proj 레이어의 출력 차원을 4(레이블 수)로 조정
import torch.nn as nn

model.classifier.out_proj = nn.Linear(in_features=model.classifier.dense.out_features, out_features=4)

# 모델의 num_labels 속성 업데이트
model.num_labels = 4


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
model

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [ ]:
# CSV 파일 로드
df = pd.read_csv("han_rename.csv", encoding="utf-8")
# NaN 값이나 None을 포함하는 행 제거
df.dropna(subset=['text', 'label'], inplace=True)

# 레이블 인코딩
label_dict = {'정치': 0, '사회': 1, '경제': 2, '국제': 3}
df['label'] = df['label'].map(label_dict)

# 훈련 데이터와 검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(df['text'], df['label'], test_size=0.1, random_state=42)


# 한글자 처리 코드

In [5]:

def remove_single_characters(text):
    # 공백을 기준으로 텍스트를 단어로 분리
    words = text.split()
    # 1글자가 아닌 단어만 선택
    filtered_words = [word for word in words if len(word) > 1]
    # 다시 텍스트로 결합
    return ' '.join(filtered_words)


In [6]:
from transformers import AutoTokenizer

# 사전 훈련된 모델에 맞는 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("copycats/koelectra-base-v3-generalized-sentiment-analysis")


# 훈련 데이터와 검증 데이터에 함수 적용
X_train_filtered = X_train.apply(remove_single_characters)
X_val_filtered = X_val.apply(remove_single_characters)

# 필터링된 데이터에 대해 토크나이징 수행 (attention_mask 포함)
train_encodings = tokenizer(X_train_filtered.tolist(), truncation=True, padding=True, max_length=512, return_tensors='pt')
val_encodings = tokenizer(X_val_filtered.tolist(), truncation=True, padding=True, max_length=512, return_tensors='pt')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

NameError: name 'X_train' is not defined

In [ ]:

# TensorDataset 생성
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(y_train.values))
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], torch.tensor(y_val.values))

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)


In [ ]:
!pip install pandas tqdm


# 본코드

In [ ]:
from torch.optim import AdamW
from tqdm.auto import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)
for epoch in range(7):  # 에포크 수
    model.train()
    for batch in tqdm(train_loader):
        # GPU로 데이터 이동
        batch = tuple(t.to(device) for t in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],  # attention_mask 추가
            'labels': batch[2]  # 수정된 레이블 인덱스
        }
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs.loss  # loss 수정
        loss.backward()
        optimizer.step()

    model.eval()
    predictions, true_vals = [], []
    for batch in val_loader:
        batch = tuple(t.to(device) for t in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1]  # attention_mask 추가
        }
        with torch.no_grad():
            outputs = model(**inputs)

        logits = outputs.logits  # logits 수정
        logits = logits.detach().cpu().numpy()
        label_ids = batch[2].cpu().numpy()  # 수정된 레이블 인덱스
        predictions.append(np.argmax(logits, axis=1).flatten())
        true_vals.append(label_ids.flatten())

    predictions = np.concatenate(predictions)
    true_vals = np.concatenate(true_vals)
    val_acc = accuracy_score(true_vals, predictions)
    print(f"Validation Accuracy: {val_acc}")


# 모델 평가부분, 전처리부분도 동봉.

In [ ]:
import pandas as pd
import numpy as np
import re

def preprocess_text(text):
    # 이메일 주소 제거
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
    # http(s) 또는 www로 시작하는 링크 제거
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    # 유니코드 색상 코드 제거
    text = re.sub(r'#[0-9a-fA-F]{6}', '', text)
    # 숫자, 문자를 제외한 모든 것 제거 (이모티콘, 한자, 특수문자 등)
    text = re.sub(r'[^\w\s]', '', text)
    # 괄호 제거
    text = re.sub(r'[\(\)\·]', '', text)
    # 연속된 마침표 제거
    text = re.sub(r'\.{2,}', '', text)
    # "기자" 단어 제거
    text = re.sub(r'기자', '', text)
    # 숫자를 찾아서 제거
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[연년월일]', '', text)

    return text


def remove_japanese_characters(text):
    # 일본어 문자 범위를 사용하여 일본어를 제거
    # 유니코드 범위:
    #   - 히라가나: 3040-309F
    #   - 카타카나: 30A0-30FF
    #   - 한자: 4E00-9FAF (일부 한자는 다른 범위에 속할 수 있음)
    # 이외에도 여러 범위가 더 있지만, 기본적인 예제로는 이 정도가 적당함
    cleaned_text = re.sub(r'[\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FAF]+', '', text)
    return cleaned_text

# 예를 들어, df가 다음과 같이 정의되어 있다고 가정합니다.
# df = pd.DataFrame({'text': ['이것은 예시 문장입니다. https://example.com', np.nan, 123, '또 다른 예시 문장입니다.']})

from transformers import AutoTokenizer

# 사전 훈련된 모델에 맞는 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("copycats/koelectra-base-v3-generalized-sentiment-analysis")


def remove_single_characters(text):
    # 공백을 기준으로 텍스트를 단어로 분리
    words = text.split()
    # 1글자가 아닌 단어만 선택
    filtered_words = [word for word in words if len(word) > 1]
    # 다시 텍스트로 결합
    return ' '.join(filtered_words)


In [ ]:
def predict_with_model(model, tokenizer, text):
    # 사용자 입력 텍스트 전처리
    text = preprocess_text(text)
    text = remove_japanese_characters(text)
    text = remove_single_characters(text)
    # 토크나이징
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # attention_mask 추가
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # 모델 예측
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    # 예측 결과
    predictions = torch.argmax(logits, dim=1)
    return predictions.cpu().numpy()

# 사용자 입력 받기
text_input = input("분류하고 싶은 텍스트를 입력하세요: ")

# 예측 실행
predicted_label_index = predict_with_model(model, tokenizer, text_input)
predicted_label = list(label_dict.keys())[list(label_dict.values()).index(predicted_label_index[0])]
print(f"입력한 텍스트의 예측 레이블: {predicted_label}")


분류하고 싶은 텍스트를 입력하세요: 저소득·저신용 서민을 위한 중금리 금융상품인 새희망홀씨 대출이 지난해 3조3천억원 공급돼 전년 실적(2조3천억원) 대비 42.3% 증가한 것으로 집계됐다.  금융감독원은 지난해 은행권의 새희망홀씨 대출 공급은 3조3천억원으로 2010년 상품 출시 뒤 모두 253만여명에게 총액 34조7천억원이 누적 공급됐다고 25일 밝혔다. 지난해 대출 대상 범위를 확대하고 신규 대출금리를 인하한데 힘입어 공급이 크게 확대될 수 있었다고 금감원은 평가했다.  지난해 은행권은 대출 범위를 ‘연소득 4000만원 이하’ 또는 ‘연소득 5000만원 이하에 신용점수 하위 20% 이하’로 넓혀 공급 규모를 확대했다. 또 2023년 연평균 금리 7.9% 수준을 유지해 일반 가계대출 신규취급액 가중평균금리(6.6%)와 격차를 1.3%포인트 수준으로 유지했다고 한다.  광고 새희망홀씨 대출은 서민층 자금 지원을 위해 2010년 도입된 상품으로 ‘연소득 3500만원 이하’ 또는 ‘연소득 4500만원 이하에 신용점수 하위 20% 이하’를 대상으로 최대 10.5% 금리로 공급되는 은행권 대출상품이다. 금감원은 2024년 새희망홀씨 공급 목표를 지난해보다 1300억원 늘린 4조1천억원으로 설정했다. 금감원은 “서민·취약차주의 접근성을 제고하고 금리 부담을 완화하기 위한 노력을 지속할 예정”이라고 밝혔다.  노현웅 기자 goloke@hani.co.kr  노현웅 기자
입력한 텍스트의 예측 레이블: 경제


# 함수화


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from transformers import ElectraForSequenceClassification, ElectraTokenizer
from torch.optim import AdamW
from tqdm.auto import tqdm


import torch.nn as nn


# 사전 훈련된 모델 로드
model = ElectraForSequenceClassification.from_pretrained("copycats/koelectra-base-v3-generalized-sentiment-analysis")
# classifier의 out_proj 레이어의 출력 차원을 4(레이블 수)로 조정

model.classifier.out_proj = nn.Linear(in_features=model.classifier.dense.out_features, out_features=4)

# 모델의 num_labels 속성 업데이트
model.num_labels = 4


# 사전 훈련된 모델 로드
tokenizer = AutoTokenizer.from_pretrained("copycats/koelectra-base-v3-generalized-sentiment-analysis")


# CSV 파일 로드
df = pd.read_csv("han_rename.csv", encoding="utf-8")
# NaN 값이나 None을 포함하는 행 제거
df.dropna(subset=['text', 'label'], inplace=True)

from transformers import AutoTokenizer



def preprocessing(df,tokenizer):
  # 레이블 인코딩
  label_dict = {'정치': 0, '사회': 1, '경제': 2, '국제': 3}
  df['label'] = df['label'].map(label_dict)

  # 훈련 데이터와 검증 데이터 분리
  X_train, X_val, y_train, y_val = train_test_split(df['text'], df['label'], test_size=0.1, random_state=42)

  # 훈련 데이터와 검증 데이터에 함수 적용
  X_train_filtered = X_train.apply(remove_single_characters)
  X_val_filtered = X_val.apply(remove_single_characters)

  # 필터링된 데이터에 대해 토크나이징 수행 (attention_mask 포함)
  train_encodings = tokenizer(X_train_filtered.tolist(), truncation=True, padding=True, max_length=512, return_tensors='pt')
  val_encodings = tokenizer(X_val_filtered.tolist(), truncation=True, padding=True, max_length=512, return_tensors='pt')

  # TensorDataset 생성
  train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(y_train.values))
  val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], torch.tensor(y_val.values))

  # DataLoader 생성
  train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=8)

  return train_loader, val_loader


# 트레이닝 단계를 위한 함수
def train_model(model, train_loader, optimizer, device):
    model.train()
    for batch in tqdm(train_loader):
        batch = tuple(t.to(device) for t in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# 검증 단계를 위한 함수
def evaluate_model(model, val_loader, device):
    model.eval()
    predictions, true_vals = [], []
    for batch in val_loader:
        batch = tuple(t.to(device) for t in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1]
        }
        with torch.no_grad():
            outputs = model(**inputs)

        logits = outputs.logits
        logits = logits.detach().cpu().numpy()
        label_ids = batch[2].cpu().numpy()
        predictions.append(np.argmax(logits, axis=1).flatten())
        true_vals.append(label_ids.flatten())

    predictions = np.concatenate(predictions)
    true_vals = np.concatenate(true_vals)
    val_acc = accuracy_score(true_vals, predictions)
    return val_acc

# 메인 트레이닝 함수
def main_train_loop(model, train_loader, val_loader, device, epochs=7):
    optimizer = AdamW(model.parameters(), lr=5e-5)

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        train_model(model, train_loader, optimizer, device)
        val_acc = evaluate_model(model, val_loader, device)
        print(f"Validation Accuracy: {val_acc}")



Epoch 1/7


  0%|          | 0/68 [00:00<?, ?it/s]

Validation Accuracy: 0.6
Epoch 2/7


  0%|          | 0/68 [00:00<?, ?it/s]

Validation Accuracy: 0.7333333333333333
Epoch 3/7


...

Validation Accuracy: 0.7166666666666667
Epoch 6/7


  0%|          | 0/68 [00:00<?, ?it/s]

Validation Accuracy: 0.7666666666666667
Epoch 7/7


  0%|          | 0/68 [00:00<?, ?it/s]

Validation Accuracy: 0.8333333333333334


# 클래스 화


In [7]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from transformers import ElectraForSequenceClassification, ElectraTokenizer
from torch.optim import AdamW
from tqdm.auto import tqdm
import os

import torch.nn as nn


def remove_single_characters(text):
    # 공백을 기준으로 텍스트를 단어로 분리
    words = text.split()
    # 1글자가 아닌 단어만 선택
    filtered_words = [word for word in words if len(word) > 1]
    # 다시 텍스트로 결합
    return ' '.join(filtered_words)

class DataPreprocessor:
    def __init__(self, tokenizer,label_dict:dict = {'정치': 0, '사회': 1, '경제': 2, '국제': 3}):
        self.tokenizer = tokenizer
        self.label_dict = label_dict

    def encode_labels(self, df):
        df['label'] = df['label'].map(self.label_dict)
        return df

    def split_data(self, df):
        return train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

    def preprocess_and_tokenize(self, texts):
        #texts_filtered = texts.apply(remove_single_characters)  # 이전에 정의한 함수 사용
        encodings = self.tokenizer(texts.tolist(), truncation=True, padding=True, max_length=512, return_tensors='pt')
        return encodings

    def create_data_loader(self, df):
        df = self.encode_labels(df)
        X_train, X_val, y_train, y_val = self.split_data(df)

        train_encodings = self.preprocess_and_tokenize(X_train)
        val_encodings = self.preprocess_and_tokenize(X_val)

        train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(y_train.values))
        val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], torch.tensor(y_val.values))

        train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=8)

        return train_loader, val_loader


class ModelTrainer:
    def __init__(self, model, device):
        self.model = model
        self.device = device
        self.model_save_index = 0
    def train(self, train_loader, optimizer):
        self.model.train()
        for batch in tqdm(train_loader):
            batch = tuple(t.to(self.device) for t in batch)
            inputs = {
                'input_ids': batch[0],
                'attention_mask': batch[1],
                'labels': batch[2]
            }
            optimizer.zero_grad()
            outputs = self.model(**inputs)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

    def evaluate(self, val_loader):
        self.model.eval()
        predictions, true_vals = [], []
        for batch in val_loader:
            batch = tuple(t.to(self.device) for t in batch)
            inputs = {
                'input_ids': batch[0],
                'attention_mask': batch[1]
            }
            with torch.no_grad():
                outputs = self.model(**inputs)

            logits = outputs.logits
            logits = logits.detach().cpu().numpy()
            label_ids = batch[2].cpu().numpy()
            predictions.append(np.argmax(logits, axis=1).flatten())
            true_vals.append(label_ids.flatten())

        predictions = np.concatenate(predictions)
        true_vals = np.concatenate(true_vals)
        return accuracy_score(true_vals, predictions)

    def run_training_loop(self, train_loader, val_loader, epochs=7):
        optimizer = AdamW(self.model.parameters(), lr=5e-5)

        for epoch in range(epochs):
          print(f"Epoch {epoch+1}/{epochs}")
          self.train(train_loader, optimizer)
          val_acc = self.evaluate(val_loader)
          print(f"Validation Accuracy: {val_acc}")
          filename = f"not_one_koelectra{self.model_save_index}.pt"
          self.save_model(os.path.join('pt_dataset', filename))
          self.model_save_index += 1  # 파일 이름이 중복되지 않도록 인덱스 증가
    def save_model(self, file_path):
      # 모델 상태 저장
      torch.save(self.model.state_dict(), file_path)
      print(f'Model saved to {file_path}')


# 사전 훈련된 모델 로드
model = ElectraForSequenceClassification.from_pretrained("copycats/koelectra-base-v3-generalized-sentiment-analysis")
# classifier의 out_proj 레이어의 출력 차원을 4(레이블 수)로 조정

model.classifier.out_proj = nn.Linear(in_features=model.classifier.dense.out_features, out_features=4)

# 모델의 num_labels 속성 업데이트
model.num_labels = 4


# 사전 훈련된 모델 로드
tokenizer = AutoTokenizer.from_pretrained("copycats/koelectra-base-v3-generalized-sentiment-analysis")

label_dict = {'정치': 0, '사회': 1, '경제': 2, '국제': 3}


# CSV 파일 로드
df = pd.read_csv("data_processed.csv", encoding="utf-8")
# NaN 값이나 None을 포함하는 행 제거
df.dropna(subset=['text', 'label'], inplace=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
preprocessor = DataPreprocessor(tokenizer,label_dict)
train_loader, val_loader = preprocessor.create_data_loader(df)
trainer = ModelTrainer(model, device)
trainer.run_training_loop(train_loader, val_loader,epochs=15)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/452M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

...

Validation Accuracy: 0.9166666666666666
Model saved to pt_dataset/not_one_koelectra12.pt
Epoch 14/15


  0%|          | 0/192 [00:00<?, ?it/s]

Validation Accuracy: 0.8411458333333334
Model saved to pt_dataset/not_one_koelectra13.pt
Epoch 15/15


  0%|          | 0/192 [00:00<?, ?it/s]

Validation Accuracy: 0.2552083333333333
Model saved to pt_dataset/not_one_koelectra14.pt


# 예측 함수 class 버전

In [ ]:
import re
import torch

class TextPreprocessor:
    def __init__(self):
        pass

    def remove_single_characters(self, text):
        words = text.split()
        filtered_words = [word for word in words if len(word) > 1]
        return ' '.join(filtered_words)

    def preprocess_text(self, text):
        text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
        text = re.sub(r'https?://\S+|www\.\S+', '', text)
        text = re.sub(r'#[0-9a-fA-F]{6}', '', text)
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'[\(\)\·]', '', text)
        text = re.sub(r'\.{2,}', '', text)
        text = re.sub(r'기자', '', text)
        text = re.sub(r'\d+', '', text)
        text = re.sub(r'[연년월일]', '', text)
        return text

    def remove_japanese_characters(self, text):
        cleaned_text = re.sub(r'[\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FAF]+', '', text)
        return cleaned_text

    def full_preprocess(self, text):
        text = self.preprocess_text(text)
        text = self.remove_japanese_characters(text)
        text = self.remove_single_characters(text)
        return text
class ModelPredictor:
    def __init__(self, model, tokenizer, device='cpu'):
        self.model = model.to(device)  # 모델을 명시적으로 디바이스로 이동
        self.tokenizer = tokenizer
        self.device = device

    def predict(self, text):
        text_preprocessor = TextPreprocessor()
        preprocessed_text = text_preprocessor.full_preprocess(text)

        inputs = self.tokenizer(preprocessed_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        input_ids = inputs['input_ids'].to(self.device)  # 입력 데이터를 모델과 같은 디바이스로 이동
        attention_mask = inputs['attention_mask'].to(self.device)  # 입력 데이터를 모델과 같은 디바이스로 이동

        with torch.no_grad():
            outputs = self.model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

        predictions = torch.argmax(logits, dim=1)
        return predictions.cpu().numpy()

# 사용 예시
# model = ...
# tokenizer = ...

text_input = input("분류하고 싶은 텍스트를 입력하세요: ")
predictor = ModelPredictor(model, tokenizer)
predicted_label_index = predictor.predict(text_input)
predicted_label = list(label_dict.keys())[list(label_dict.values()).index(predicted_label_index[0])]
print(f"입력한 텍스트의 예측 레이블: {predicted_label}")


분류하고 싶은 텍스트를 입력하세요: ㅇㅇ
입력한 텍스트의 예측 레이블: 국제
